In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
def swish(x):
    x*tf.nn.sigmoid(x)

In [2]:
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.convnext import ConvNeXtTiny

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

In [5]:
x_train=tf.keras.applications.convnext.preprocess_input(x_train)
x_test=tf.keras.applications.convnext.preprocess_input(x_test)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model_tiny=ConvNeXtTiny(
    include_top=False,
    weights='imagenet',
    input_shape=(64,64,3)
)

In [9]:
base_model_tiny.trainable=False

In [10]:
model=models.Sequential([
    base_model_tiny,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='swish'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ convnext_tiny (Functional)           │ (None, 2, 2, 768)           │      27,820,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 768)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         787,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,296,580 (111.76 MB)

 Trainable params: 1,472,868 (5.62 MB)

 Non-trainable params: 27,823,712 (106.14 MB)

In [11]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [12]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [13]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [14]:
import time
start_time=time.time()

In [15]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [16]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1172/1172 ━━━━━━━━━━━━━━━━━━━━ 77s 53ms/step - accuracy: 0.0551 - loss: 4.8466 - val_accuracy: 0.3892 - val_loss: 2.5834
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 42ms/step - accuracy: 0.2353 - loss: 3.3095 - val_accuracy: 0.4582 - val_loss: 2.1419
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 42ms/step - accuracy: 0.3046 - loss: 2.8312 - val_accuracy: 0.4954 - val_loss: 1.9339
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 42ms/step - accuracy: 0.3463 - loss: 2.6109 - val_accuracy: 0.5215 - val_loss: 1.8058
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 41ms/step - accuracy: 0.3840 - loss: 2.4171 - val_accuracy: 0.5340 - val_loss: 1.7209
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 41ms/step - accuracy: 0.3940 - loss: 2.3246 - val_accuracy: 0.5498 - val_loss: 1.6567
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 41ms/step - accuracy: 0.4192 - loss: 2.2076 - val_accuracy: 0.5557 - val_loss: 1.6165
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 49s 42ms/step - accuracy: 0.4336 - loss: 2.13

In [17]:
head_training_time=time.time()-start_time

In [18]:
base_model_tiny.trainable=True

In [19]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [20]:
start_fine_tune=time.time()

In [21]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 123s 67ms/step - accuracy: 0.5657 - loss: 1.5519 - val_accuracy: 0.6932 - val_loss: 1.0559
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 54s 46ms/step - accuracy: 0.6313 - loss: 1.2952 - val_accuracy: 0.7158 - val_loss: 0.9594
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 53s 45ms/step - accuracy: 0.6563 - loss: 1.1846 - val_accuracy: 0.7309 - val_loss: 0.9074
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 52s 45ms/step - accuracy: 0.6708 - loss: 1.1313 - val_accuracy: 0.7426 - val_loss: 0.8669
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 54s 46ms/step - accuracy: 0.6984 - loss: 1.0326 - val_accuracy: 0.7516 - val_loss: 0.8354
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 52s 44ms/step - accuracy: 0.7060 - loss: 1.0042 - val_accuracy: 0.7570 - val_loss: 0.8128
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 52s 44ms/step - accuracy: 0.7256 - loss: 0.9386 - val_accuracy: 0.7651 - val_loss: 0.7854
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 52s 44ms/step - accuracy: 0.7451 

In [22]:
fine_tune_time=time.time()-start_fine_tune

In [23]:
test_loss, test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")


Test Accuracy: 77.84%
Total training time: 1610.24 seconds


In [24]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7845 - loss: 0.7357
Validation accuracy: 77.8080%


In [25]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step


In [26]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for ConvNext tiny:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for ConvNext tiny:
Accuracy: 0.7784
Precision: 0.7806
Recall: 0.7784
F1-Score: 0.7766


In [31]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["ConvNext Tiny"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [32]:
print(results)

           Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  ConvNext Tiny         0.7784              0.77808   0.747812   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             1012.746247            597.494572      29296580  


In [33]:
results.to_csv("final_results.csv",index=False)
print("\nResults saved to final_results.csv")


Results saved to final_results.csv
